# 수정사항
- 이중루프에 의해 겹치는 오브젝트 제거하는 로직의 오류를 수정했습니다
  - i, j 인덱스 비교시 i != j 가 아니라 i == j 로 비교해야 합니다
  - 자기 자신일 경우 건너뛰는 로직이기 때문입니다

- border 영역을 세그멘테이션 영역으로 처리하는 오류를 수정했습니다
  - border 영역이 본 오브젝트 영역보다 더 크기 때문에 발생한 오류입니다
  - border 색상 값을 확인하고 해당 색상 조건일 경우 검정색 처리하는 로직으로 수정했습니다

- 이미지 1장 증강하는 스크립트의 파일명이 고유하도록 수정했습니다
  - 오브젝트 이미지와 배경 이미지의 이름을 조합하여 고유한 이름이 되도록 했습니다

- 모든 가능한 조합으로 트레이닝 데이터세트를 대량생산하는 구문을 추가했습니다
  - 이것을 이용해서 Chap06 에서 사용합니다

# Chap05 이미지 대량 생산
- Chap04에서 돌리던거 마저 돌리면 대량으로 생산되기는 하겠지요
- 그러나 여기서 말하는 대량 생산이란 이미지 생산 계획을 세우고 그에 맞게 증강을 수행하는 것을 의미합니다
- 약간의 bash shell 커맨드를 이용하는 방법도 소개하고자 합니다
- 여기서 소개하는 방법은 반드시 해야 하는 방법은 아니며 하나의 예시로서 제시하고자 합니다
- 그럼 VOC2012 데이터셋부터 다시 시작해봅시다

In [ ]:
! wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
! tar xvf VOCtrainval_11-May-2012.tar

# 아래 커맨드를 이용하면 python을 쓰지 않고도 오브젝트의 분포를 파악할 수 있습니다
- 주석을 위에서부터 하나씩 풀면서 결과를 확인해봅시다
- person 의 비율이 지나치게 높은 것을 확인할 수 있습니다

In [ ]:
# ! find VOCdevkit/VOC2012/Annotations -name "*.xml" 
# ! find VOCdevkit/VOC2012/Annotations -name "*.xml" | xargs grep "<name" 
! find VOCdevkit/VOC2012/Annotations -name "*.xml" | xargs grep "<name" | awk '{print $2}' | sort | uniq -c

# 잠깐만요, Main task 데이터세트가 아니군요
- Mak task 데이터세트가 있는 train.txt로 다시 해봅시다
- 여전히 밸런스가 맞지 않습니다

In [ ]:
! cat /content/VOCdevkit/VOC2012/ImageSets/Main/train.txt | awk '{printf "VOCdevkit/VOC2012/Annotations/%s.xml\n",$1}' | xargs grep "<name" | awk '{print $2}' | sed 's/<name>//g' | sed 's/<\/name>//g'  | grep -v hand | grep -v head | grep -v foot |  sort | uniq -c

# 오브젝트의 크기 분포를 클래스별로 확인해봅시다
- YOLO 모델 학습을 예시로 사용할 것이기 때문에 anchor 개념이 들어갑니다
- 따라서 이미지 내의 오브젝트 크기가 anchor 변수에 영향을 줍니다
- 이것도 쉘스크립트로 가능할까요?
  - 빠른 포기하고 python 으로 넘어갑니다
  - 어떤 방법이든 효율적으로 할 수 있는 방향으로 갑니다

In [ ]:
! find VOCdevkit/VOC2012/Annotations -name "*.xml" | xargs grep "<width\|<height\|<xmin\|<ymin\|<xmax\|<ymax\|<name" | head -n 100

### VOC 2012 Main task 의 train 데이터세트에 대해 클래스별 오브젝트 크기 분포 확인

In [ ]:
import cv2
import sys
import os
import numpy as np
import xml.etree.ElementTree as Et
import matplotlib.pyplot as plt

colab_env = True

try:
    from google.colab.patches import cv2_imshow
except:
    colab_env = False

ANNOTATION_BASE = '/content/VOCdevkit/VOC2012/Annotations'
ANNOTATION_SUFFIX = '.xml'
train_txt_filename = '/content/VOCdevkit/VOC2012/ImageSets/Main/train.txt'
train_txt_file = open(train_txt_filename, 'r')
train_files_list = train_txt_file.readlines()
train_files = [t.strip() for t in train_files_list]

object_stat = {}

for train_file in train_files:
    annotation_file = os.path.join(ANNOTATION_BASE, train_file) + ANNOTATION_SUFFIX

    tree = Et.parse(annotation_file)
    root = tree.getroot()
    objects = root.findall("object")
    size = root.find("size")
    width = int(size.find("width").text)
    height = int(size.find("height").text)

    for obj in objects:
        name = obj.find("name").text
        bndbox = obj.find("bndbox")
        xmin = int(bndbox.find("xmin").text)
        ymin = int(bndbox.find("ymin").text)
        xmax = int(bndbox.find("xmax").text)
        ymax = int(bndbox.find("ymax").text)    
        area = float((xmax - xmin) * (ymax - ymin)) / float(width * height)
        if name not in object_stat:
            object_stat[name] = []
        object_stat[name].append(area)
        # print(object_stat)



for name, areas in object_stat.items():
    # print(name, area)
    plt.figure(figsize=(8, 4))
    plt.hist(areas, bins=20)
    plt.gca().set(title='{} Histogram'.format(name), ylabel='Frequency')
    plt.ylim(0, 1000)
    plt.show()    
    print()

# 아래와 같은 3단계로 진행해보고자 합니다

## 1. 소스 오브젝트와 배경이미지 만들기
- 단일 오브젝트로 구성된 이미지와, segmentation mask로 이루어지는 소스 오브젝트 구성
- Annotation 기준으로 사물이 없는 배경으로만 이루어진 이미지

## 2. 소스 오브젝트와 배경이미지를 합성하는 스크립트 만들기
- 소스 오브젝트와 해당 오브젝트에 대한 segmentation mask를 이용하여 배경이미지에 합성합니다
- Chap04에 했던 내용과 동일합니다

## 3. python 스크립트를 호출하는 bash 스크립트 만들기
- 파리미터를 넘기는 bash 스크립트를 만듭니다
- 수량은 bash 스크립트로 조절합니다


# 일단 이함수는 하나 빼놓읍시다

In [ ]:
ANNOTATION_BASE = '/content/VOCdevkit/VOC2012/Annotations'
JPEGIMAGE_BASE = '/content/VOCdevkit/VOC2012/JPEGImages'
SEGMENTATION_BASE = '/content/VOCdevkit/VOC2012/SegmentationObject'

JPEGIMAGE_SUFFIX = '.jpg'
SEGMENTATION_SUFFIX = '.png'
ANNOTATION_SUFFIX = '.xml'

def display_image(image):
    if colab_env:
        cv2_imshow(image)

    else:
        cv2.imshow('image', image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

# 문제가 생겨서 새로 할때 기존 출력을 지웁니다
- -rf 옵션 사용시 삭제대상을 잘 확인하고 조심해서 사용합니다

In [ ]:
! rm -rf /content/backgrounds

## 소스 배경 만들기

In [ ]:
import cv2
import sys
import os
import numpy as np
import xml.etree.ElementTree as Et

colab_env = True

try:
    from google.colab.patches import cv2_imshow
except:
    colab_env = False


# 배경이 저장될 디렉토리 입니다(상대경로)
save_base = '/content/backgrounds'
os.makedirs(save_base, exist_ok=True)

minimum_width = 256
minimum_height = 256


train_txt_filename = '/content/VOCdevkit/VOC2012/ImageSets/Main/train.txt'
train_txt_file = open(train_txt_filename, 'r')
train_files_list = train_txt_file.readlines()
train_files = [t.strip() for t in train_files_list]

object_stat = {}

for train_file in train_files:
    org_jpg = os.path.join(JPEGIMAGE_BASE, train_file) + JPEGIMAGE_SUFFIX
    org_xml = os.path.join(ANNOTATION_BASE, train_file) + ANNOTATION_SUFFIX

    org_img = cv2.imread(org_jpg)
    org_tree = Et.parse(org_xml)
    org_root = org_tree.getroot()
    org_objects = org_root.findall("object")
    org_size = org_root.find("size")
    org_width = int(org_size.find("width").text)
    org_height = int(org_size.find("height").text)

    # 어떤 VOC2012 이미지에서 오브젝트가 하나밖에 없는게 아니라면 무시
    if len(org_objects) > 1:
        continue

    for obj in org_objects:
        # Annotation 내 오브젝트 기준으로 SegmentationObject 이미지를 탐색
        name = obj.find("name").text
        bndbox = obj.find("bndbox")
        xmin = int(bndbox.find("xmin").text)
        ymin = int(bndbox.find("ymin").text)
        xmax = int(bndbox.find("xmax").text)
        ymax = int(bndbox.find("ymax").text)    

        # print(org_width, org_height, xmin, ymin, xmax, ymax)

        f_prefix = None

        # 좌측으로 한번 자르고
        if xmin > minimum_width and org_height > minimum_height:
            background_img = org_img[:,:xmin]
            f_prefix = 'left'  

        # 우측으로 한번 자르고
        elif org_width - xmax > minimum_width and org_height > minimum_height:
            background_img = org_img[:,xmax:]    
            f_prefix = 'right'    

        # 위로 한번 자르고
        elif ymin > minimum_height and org_width > minimum_width:
            background_img = org_img[:ymin,:]
            f_prefix = 'top'   

        # 아래로 한번 자르고
        elif org_height - ymax > minimum_height and org_width > minimum_width:
            background_img = org_img[ymax:,:]
            f_prefix = 'bottom'

        if f_prefix is not None:
            # display_image(background_img)

            save_file_path = os.path.join(save_base, '{}_{}_{}.jpg'.format(f_prefix, name, train_file))
            os.makedirs(os.path.dirname(save_file_path), exist_ok=True)
            cv2.imwrite(save_file_path, background_img)
            print(org_jpg, f_prefix, '\n')

## 뭔가 이상할 때에는 디버깅을 합시다
- 꼭 디버거를 걸고 해야만 디버깅은 아닙니다

In [ ]:
import cv2
import xml.etree.ElementTree as Et

colab_env = True

try:
    from google.colab.patches import cv2_imshow
except:
    colab_env = False

debug_image = '/content/VOCdevkit/VOC2012/JPEGImages/2011_002851.jpg'
debug_xml = '/content/VOCdevkit/VOC2012/Annotations/2011_002851.xml'
img = cv2.imread(debug_image)

org_xml = open(debug_xml, "r")
org_tree = Et.parse(org_xml)
org_root = org_tree.getroot()
org_objects = org_root.findall("object")
org_size = org_root.find("size")
org_width = int(org_size.find("width").text)
org_height = int(org_size.find("height").text)


for obj in org_objects:
    # Annotation 내 오브젝트 기준으로 SegmentationObject 이미지를 탐색
    name = obj.find("name").text
    bndbox = obj.find("bndbox")
    xmin = int(bndbox.find("xmin").text)
    ymin = int(bndbox.find("ymin").text)
    xmax = int(bndbox.find("xmax").text)
    ymax = int(bndbox.find("ymax").text)
    img = cv2.rectangle(img, (xmin, ymin), (xmax, ymax), color=(255,0,0))
    img = cv2.putText(img, name, org=(xmin, ymin), fontFace=cv2.FONT_HERSHEY_COMPLEX , fontScale=1.5, color=(255,0,255))

if colab_env:
    cv2_imshow(img)

else:
    cv2.imshow(img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

## 소스 오브젝트 만들기
- 이번에는 오브젝트 주변의 상하좌우를 배경이미지로 잘라내는 것이 아니라 그 반대입니다
- 오브젝트 영역 내부를 잘라내겠습니다

In [ ]:
import cv2
import sys
import os
import numpy as np
import xml.etree.ElementTree as Et
from google.colab.patches import cv2_imshow

colab_env = True

try:
    from google.colab.patches import cv2_imshow
except:
    colab_env = False


object_save_dir = '/content/objects'
segment_save_dir = '/content/segmentations'

os.makedirs(object_save_dir, exist_ok=True)
os.makedirs(segment_save_dir, exist_ok=True)

exclude_class_list = ['person']

train_txt_filename = '/content/VOCdevkit/VOC2012/ImageSets/Main/train.txt'
train_txt_file = open(train_txt_filename, 'r')
train_files_list = train_txt_file.readlines()
train_files = [t.strip() for t in train_files_list]

object_stat = {}

f_index = 0

for train_file in train_files:

    org_jpg = os.path.join(JPEGIMAGE_BASE, train_file) + JPEGIMAGE_SUFFIX
    org_xml = os.path.join(ANNOTATION_BASE, train_file) + ANNOTATION_SUFFIX
    org_segmentations = os.path.join(SEGMENTATION_BASE, train_file) + SEGMENTATION_SUFFIX
    org_img = cv2.imread(org_jpg)

    if not os.path.exists(org_segmentations):
        continue
    org_segment = cv2.imread(org_segmentations)
    org_tree = Et.parse(org_xml)
    org_root = org_tree.getroot()
    org_objects = org_root.findall("object")
    org_size = org_root.find("size")
    org_width = int(org_size.find("width").text)
    org_height = int(org_size.find("height").text)

    # 이번에는 이미지 내에 여러 오브젝트가 있다면 해당 오브젝트들을 모두 개별 이미지로 잘라내면 됩니다
    
    for obj in org_objects:
        # Annotation 내 오브젝트 기준으로 SegmentationObject 이미지를 탐색
        name = obj.find("name").text
        bndbox = obj.find("bndbox")
        xmin = int(bndbox.find("xmin").text)
        ymin = int(bndbox.find("ymin").text)
        xmax = int(bndbox.find("xmax").text)
        ymax = int(bndbox.find("ymax").text)

        # 이 구문은 일단 주석처리된 상태로 돌려봅시다
        if (obj.find("occluded") is not None and obj.find("occluded").text == '1') or (obj.find("truncated") is not None and obj.find("truncated").text == '1'):
            continue

        cropped_image = org_img[ymin:ymax, xmin:xmax]
        # display_image(cropped_image)

        save_file_path = '{}/{}/{}_{}_{}.jpg'.format(object_save_dir, name, f_index, name, train_file) 
        os.makedirs(os.path.dirname(save_file_path), exist_ok=True)
        cv2.imwrite(save_file_path, cropped_image)

        cropped_segment = org_segment[ymin:ymax, xmin:xmax]
        # display_image(cropped_segment)

        save_file_path = '{}/{}/{}_{}_{}.png'.format(segment_save_dir, name, f_index, name, train_file)
        os.makedirs(os.path.dirname(save_file_path), exist_ok=True)
        cv2.imwrite(save_file_path, cropped_segment)        
        
        print(f_index, name, train_file)

        f_index = f_index + 1

# 문제가 생겨서 새로 할때 기존 출력을 지웁니다
- -rf 옵션 사용시 삭제대상을 잘 확인하고 조심해서 사용합니다

In [ ]:
! rm -rf /content/objects
! rm -rf /content/segmentations

## 뭔가 또 불편합니다
- 지정한 오브젝트 외에 함께 들어있는 오브젝트가 존재하지요
- 오브젝트 영역 내에 다른 오브젝트가 없는 케이스로 한정해봅시다


## 2가지 접근방법
### 1번
- Annotation 정보 이용하기
  - occluded = 0 or null

### 2번
- 이중루프 돌려서 걸러내기
  - Annotation 에서 지원하지 않아도 가능한 부분

## 그 전에 IoU 라는 것을 생각합시다
- https://en.wikipedia.org/w/index.php?title=Intersection_over_union&redirect=no
- Intersection Over Union = 0 인 오브젝트
- 하나의 이미지 안에서 자신을 제외한 모든 다른 오브젝트와 겹치는 영역이 없는
  - IoU가 0인 오브젝트가 우리가 원하는 대상입니다
- IoU 는 어떻게 계산하죠?
  - 2개의 영역에 대한 intersection 을 구합니다
  - 2개의 영역에 대한 union 을 구합니다
  - intersection 나누기 union 을 리턴합니다
- 그러면 그냥 여기서는 intersection이 0인것만 찾으면 되는거 아니에요?
  - 네 맞아요^^;;
  - 그러나 Object detection 에서 IoU는 항상 등장하는 개념이라 한번 짚고 넘어가 봅니다
  - Object의 ground truth 와 predicted area가 얼마나 ground truth에 근접하는지 설명하는 수치이기 때문입니다

In [ ]:
from xml.etree.ElementTree import Element

def get_iou(bndbox_i:Element, bnxbox_j:Element):  
    xmin_i = int(bndbox_i.find("xmin").text)
    ymin_i = int(bndbox_i.find("ymin").text)
    xmax_i = int(bndbox_i.find("xmax").text)
    ymax_i = int(bndbox_i.find("ymax").text)
    box1 = [xmin_i, ymin_i, xmax_i, ymax_i]

    xmin_j = int(bndbox_j.find("xmin").text)
    ymin_j = int(bndbox_j.find("ymin").text)
    xmax_j = int(bndbox_j.find("xmax").text)
    ymax_j = int(bndbox_j.find("ymax").text)
    box2 = [xmin_j, ymin_j, xmax_j, ymax_j]

    intersection = get_intersection(box1, box2)
    union = get_area(box1) + get_area(box2) - intersection
    iou = float(intersection) / float(union)

    return intersection


def get_intersection(box1: list, box2: list):
    box1_xmin = box1[0]
    box1_ymin = box1[1]
    box1_xmax = box1[2]
    box1_ymax = box1[3]

    box2_xmin = box2[0]
    box2_ymin = box2[1]
    box2_xmax = box2[2]
    box2_ymax = box2[3]

    xmin = box1_xmin if box1_xmin > box2_xmin else box2_xmin
    ymin = box1_ymin if box1_ymin > box2_ymin else box2_ymin
    xmax = box1_xmax if box1_xmax < box2_xmax else box2_xmax
    ymax = box1_ymax if box1_ymax < box2_ymax else box2_ymax

    box = [xmin, ymin, xmax, ymax]

    return get_area(box)

def get_area(box: list):
    xmin = box[0]
    ymin = box[1]
    xmax = box[2]
    ymax = box[3]

    return (xmax - xmin) * (ymax - ymin)  

## 함수 작성을 했으니 이제 이를 이용하는 이중루프 돌려봅시다
- 동일한 objects array를 이중루프를 돌립니다
- 자기 자신을 제외한 나머지 모든 오브젝트와 IoU를 비교하는 것입니다
- IoU가 0보다 크면, 즉 겹치는 영역이 조금이라도 있으면 오브젝트 추출에서 제외합니다
- IoU가 정확히 0이면, 즉 겹치는 영역이 조금도 없으면 해당 오브젝트를 추출합니다

In [ ]:
! rm -rf /content/objects3

In [ ]:
import cv2
import sys
import os
import numpy as np
import xml.etree.ElementTree as Et
from xml.etree.ElementTree import Element
from google.colab.patches import cv2_imshow

colab_env = True

try:
    from google.colab.patches import cv2_imshow
except:
    colab_env = False

# 단일 오브젝트 이미지가 저장될 디렉토리 입니다(상대경로)
object_save_dir = '/content/objects2'
segment_save_dir = '/content/segmentations2'

exclude_class_list = ['person']


train_txt_filename = '/content/VOCdevkit/VOC2012/ImageSets/Main/train.txt'
train_txt_file = open(train_txt_filename, 'r')
train_files_list = train_txt_file.readlines()
train_files = [t.strip() for t in train_files_list]

object_stat = {}

f_index = 0

for train_file in train_files:

    org_jpg = os.path.join(JPEGIMAGE_BASE, train_file) + JPEGIMAGE_SUFFIX
    org_xml = os.path.join(ANNOTATION_BASE, train_file) + ANNOTATION_SUFFIX
    org_segmentations = os.path.join(SEGMENTATION_BASE, train_file) + SEGMENTATION_SUFFIX
    org_img = cv2.imread(org_jpg)

    if not os.path.exists(org_segmentations):
        continue
    org_segment = cv2.imread(org_segmentations)
    org_tree = Et.parse(org_xml)
    org_root = org_tree.getroot()
    org_objects = org_root.findall("object")
    org_size = org_root.find("size")
    org_width = int(org_size.find("width").text)
    org_height = int(org_size.find("height").text)

    # 이번에는 이미지 내에 여러 오브젝트가 있다면 해당 오브젝트들을 모두 개별 이미지로 잘라내면 됩니다
    for i, obj_i in enumerate(org_objects):  
        name = obj_i.find("name").text

        if name in exclude_class_list:
            # print('excluding {}'.format(name))
            continue
        bndbox_i = obj_i.find("bndbox")  
        no_intersection = True
        for j, obj_j in enumerate(org_objects):
            bndbox_j = obj_j.find("bndbox")            

            # 자기 자신은 건너뜁니다 i != j가 아니라 i == j 로 비교해야 합니다
            # 자기 자신이 아닌 오브젝트와 비교했을때 겹치는 영역이 있으면 건너뜁니다
            if i == j or get_iou(bndbox_i, bndbox_j) > 0:
                no_intersection = False
                # print('intersection exists, skip')
                break
            
            # 자기 자신이 아닌 오브젝트와 비교했을때 겹치는 영역이 없을 때에 한해 진행합니다
            if no_intersection:
                xmin = int(bndbox_i.find("xmin").text)
                ymin = int(bndbox_i.find("ymin").text)
                xmax = int(bndbox_i.find("xmax").text)
                ymax = int(bndbox_i.find("ymax").text)

                # 단일 오브젝트 이미지를 잘라냅니다
                cropped_image = org_img[ymin:ymax, xmin:xmax]
                # display_image(cropped_image)

                # 단일 오브젝트 이미지에 대응되는 세그먼트를 잘라냅니다
                cropped_segment = org_segment[ymin:ymax, xmin:xmax]
                # display_image(cropped_segment)

                # 잘라난 단일 오브젝트 이미지를 저장합니다                    
                save_file_path = '{}/{}/{}_{}_{}.jpg'.format(object_save_dir, name, f_index, name, train_file) 
                os.makedirs(os.path.dirname(save_file_path), exist_ok=True)
                cv2.imwrite(save_file_path, cropped_image)


                # 잘라난 단일 세그먼트 이미지를 저장합니다
                save_file_path = '{}/{}/{}_{}_{}.png'.format(segment_save_dir, name, f_index, name, train_file)
                os.makedirs(os.path.dirname(save_file_path), exist_ok=True)
                cv2.imwrite(save_file_path, cropped_segment)       

                # 디버깅 용도로 print 하나 찍습니다
                print(f_index, name, train_file)

                f_index = f_index + 1
            else:
                print('This should not be shown')

# XML Annotation 의 occluded, truncated를 이용했을 때와 비교해봅니다
- 2011_002851 의 경우 segmentation 영역 기준으로는 겹치지 않지만 bndbox 기준으로는 겹칩니다
- bndbox 기준으로 이중루프 돌려서 필터링한 경우에는 아래와같이 소거됨을 확인할 수 있습니다
- 생산된 총 오브젝트 수도 더 적은 것을 확인할 수 있습니다

In [ ]:
! find objects2 -name "*2011_002851*"

# 몇개나 만들어졌는지 확인해볼까요?
- 이제부터 슬슬 bash shell을 써보도록 하겠습니다
- bash shell은 필수가 아닙니다
  - 아래 표현은 python에서도 충분히 가능합니다
  - 다만, python 에서 import os , os.walk 쓰는 시간에 bash 한줄로 가능하기 때문에 일단 이렇게 사용하겠습니다

In [ ]:
! find /content/backgrounds -name "*.jpg" | wc -l | awk  '{printf "Total background images: %s\n", $1}'

In [ ]:
! find /content/objects -name "*.jpg" | awk -F"/" '{print $4}' | sort | uniq -c | awk '{print $2, $1}'

In [ ]:
! find /content/segmentations -name "*.png" | awk -F"/" '{print $4}' | sort | uniq -c | awk '{print $2, $1}'

# 원래 목적대로 이미지 한장 만드는 함수를 만들어봅시다
- 필요한것
  - 원본 오브젝트 이미지
  - 배경
  - 세그멘테이션 마스크

- 반환하는 값
  - 합성된 이미지
  - 합성된 이미지 내의 오브젝트 좌표

In [ ]:
# YOLO annotation 생성을 위해 classes.txt 를 먼저 만듭니다
! cat /content/VOCdevkit/VOC2012/ImageSets/Main/train.txt | awk '{printf "VOCdevkit/VOC2012/Annotations/%s.xml\n",$1}' | xargs grep "<name" | awk '{print $2}' | sed 's/<name>//g' | sed 's/<\/name>//g'  | grep -v hand | grep -v head | grep -v foot |  sort | uniq > classes.txt

In [ ]:
import sys
import os
import numpy as np
import cv2
import traceback
import random
import xml.etree.ElementTree as Et

colab_env = True

try:
    from google.colab.patches import cv2_imshow
except:
    colab_env = False


def read_classes():
    with open('classes.txt', 'r') as f:
        return [t.strip() for t in f.readlines()]


def build_yolo_label(filename, class_index, augmented_image, augmented_coordinate):
    with open(filename, 'w') as f:
        bg_height, bg_width, bg_channel = augmented_image.shape

        # [[50,50],[100,100]]
        object_xmin = augmented_coordinate[0][0]
        object_ymin = augmented_coordinate[0][1]
        object_width = augmented_coordinate[1][0] - augmented_coordinate[0][0]
        object_height = augmented_coordinate[1][1] - augmented_coordinate[0][1]

        x_center = (object_xmin + object_width/2)/bg_width
        y_center = (object_ymin + object_height/2)/bg_height
        width = object_width / bg_width
        height = object_height / bg_height

        annotation = '{} {} {} {} {}'.format(class_index, x_center, y_center, width, height)
        f.write(annotation)


# 이 함수의 오류를 수정합니다
# border의 영역이 본체(?) 의 영역보다 더 클수 있습니다 /content/segmentations/bird/284_bird_2009_000454.png 의 경우가 그러합니다
# 따라서 그냥 border 영역의 색깔 [192 224 224] 을 잡아서 해당 영역을 제거하는 방식을 사용합니다
def remove_border(jpeg_image_path, segmented_image_path):    

    original_image = cv2.imread(jpeg_image_path)
    segmented_image = cv2.imread(segmented_image_path)
    segmented_image[(segmented_image == [192, 224, 224]).all(axis=2)] = [0,0,0]

    return segmented_image

    # 아래 내용은 히스토리로 남겨두고 사용하지 않습니다
    segmented_image_arr = (np.array(segmented_image)).reshape(-1,3)

    colors, counts = np.unique(segmented_image_arr, axis=0, return_counts=1)

    counts = counts.tolist()
    colors = colors.tolist()    
    black_index = colors.index([0,0,0])    

    del colors[black_index]
    del counts[black_index] 
    max_color = colors[counts.index(max(counts))]

    original_image_arr = (np.array(original_image.copy())).reshape(-1,3)

    for s, pixel in enumerate(segmented_image_arr):
        if not (pixel == max_color).all():
            segmented_image_arr[s] = [0,0,0]

    segmented_image_arr = segmented_image_arr.reshape(segmented_image.shape)

    return segmented_image_arr

# 사이즈와 위치를 랜덤하게 변화시키는 로직을 추가했습니다
def create_augmented_image(original_image_path, background_image_path, segmented_image_path):
    
    minimum_size = 0.1

    original_image = cv2.imread(original_image_path)

    segmented_image = remove_border(jpeg_image_path=original_image_path, segmented_image_path=segmented_image_path)

    background_jpg = background_image_path
    background_img = cv2.imread(background_jpg)
    
    org_height, org_width, org_channel = original_image.shape
    bg_height, bg_width, bg_channel = background_img.shape
    
    width_ratio = float(org_width) / float(bg_width)
    height_ratio = float(org_height) / float(bg_height)
    
    if height_ratio > width_ratio:
        maximum_size = height_ratio
    else:
        maximum_size = width_ratio
    
    if maximum_size > 1:
        maximum_size = 0.8
        
    size = random.uniform(minimum_size, maximum_size)    
    
    if org_height > org_width:
        new_size = ( int((float(bg_height * size) / float(org_height)) * org_width), int(bg_height * size))
    else:
        new_size = (int(bg_width * size),  int((float(bg_width * size) / float(org_width)) * org_height))
    
    original_image = cv2.resize(original_image, dsize=new_size)
    segmented_image = cv2.resize(segmented_image, dsize=new_size)                                

    background_height, background_width, background_channel = background_img.shape

    object_height, object_width, object_channel = original_image.shape

#     x_start = int((background_width - object_width)/2)
#     y_start = int((background_height - object_height)/2)
    x_start = int(random.uniform(0, float(bg_width - new_size[0]) / float(bg_width)) * bg_width)
    y_start = int(random.uniform(0, float(bg_height - new_size[1]) / float(bg_height)) * bg_height)

    new_original_image = np.zeros(background_img.shape, dtype=np.uint8)
    new_segmented_image = np.zeros(background_img.shape, dtype=np.uint8)                        
    new_original_image[y_start:y_start+object_height,x_start:x_start+object_width] = original_image        
    new_segmented_image[y_start:y_start+object_height,x_start:x_start+object_width] = segmented_image
    
#     print('size', x_start, y_start)

    new_segmented_image = cv2.cvtColor(new_segmented_image, cv2.COLOR_BGR2GRAY)

    final_image = cv2.copyTo(new_original_image, new_segmented_image, background_img)

    new_xmin = x_start
    new_ymin = y_start
    new_xmax = x_start + object_width
    new_ymax = y_start + object_height

    return final_image, [[new_xmin, new_ymin], [new_xmax, new_ymax]]

if __name__ == '__main__':
    if len(sys.argv) < 4:
        print('not enough arguments')
    else:
        classes = read_classes()

        original_image_path = sys.argv[1]
        background_image_path = sys.argv[2]
        segmented_image_path = sys.argv[3]

        # 오브젝트와 배경의 조합이므로 오브젝트 파일 이름과 배경 파일 이름의 조합으로 생성하면 고유값을 유지할 수 있습니다. split으로 확장자를 제거합니다
        original_image_filename = os.path.basename(original_image_path).split('.')[0]
        background_image_filename = os.path.basename(background_image_path).split('.')[0]

        # 지정된 클래스파일 기준으로 했을 때, 증강되는 오브젝트의 클래스 인덱스가 몇번인지 확인합니다
        classname = original_image_path.split('/')[3]
        class_index = classes.index(classname)

        # 증강 이미지와 annotation을 1장 생성합니다
        augmented_image, augmented_coordinate = create_augmented_image(original_image_path, background_image_path, segmented_image_path)
        
        if augmented_image is None:
            sys.exit(0)              

        augmented_image_filename = '{}/{}/{}/{}-{}.jpg'.format(save_dir, classname, bg_classname, original_image_filename, background_image_filename)
        os.makedirs(os.path.dirname(augmented_image_filename), exist_ok=True)
        cv2.imwrite(augmented_image_filename, augmented_image)      
        augmented_label_filename = '{}/{}/{}/{}-{}.txt'.format(save_dir, classname, bg_classname, original_image_filename, background_image_filename)
        
        build_yolo_label(augmented_label_filename, class_index, augmented_image, augmented_coordinate)
        
        print(augmented_image_filename)

In [ ]:
classes = read_classes()

def call_create_augmented_image(original_image_path, background_image_path, segmented_image_path, save_dir='augmented_images', size = 0.5):

        # 오브젝트와 배경의 조합이므로 오브젝트 파일 이름과 배경 파일 이름의 조합으로 생성하면 고유값을 유지할 수 있습니다. split으로 확장자를 제거합니다
        original_image_filename = os.path.basename(original_image_path).split('.')[0]
        background_image_filename = os.path.basename(background_image_path).split('.')[0]

        # 지정된 클래스파일 기준으로 했을 때, 증강되는 오브젝트의 클래스 인덱스가 몇번인지 확인합니다
        classname = original_image_path.split('/')[1]
        bg_classname = background_image_path.split('/')[1].split('_')[1]
        class_index = classes.index(classname)

        # 증강 이미지와 annotation을 1장 생성합니다
        augmented_image, augmented_coordinate = create_augmented_image(original_image_path, background_image_path, segmented_image_path)
        
        if augmented_image is None:
            return              

        augmented_image_filename = '{}/{}/{}/{}-{}.jpg'.format(save_dir, classname, bg_classname, original_image_filename, background_image_filename)
        os.makedirs(os.path.dirname(augmented_image_filename), exist_ok=True)
        cv2.imwrite(augmented_image_filename, augmented_image)      
        augmented_label_filename = '{}/{}/{}/{}-{}.txt'.format(save_dir, classname, bg_classname, original_image_filename, background_image_filename)
        
        build_yolo_label(augmented_label_filename, class_index, augmented_image, augmented_coordinate)
        
        print(augmented_image_filename)

# 함수 동작을 검증해봅시다

In [ ]:
augmented_image, augmented_coordinate = create_augmented_image( original_image_path = '/content/objects/bird/337_bird_2009_001385.jpg', \
    background_image_path = '/content/backgrounds/bottom_aeroplane_2009_002314.jpg', \
    segmented_image_path = '/content/segmentations/bird/337_bird_2009_001385.png')

display_image(augmented_image)
augmented_image_image_with_label = cv2.rectangle(augmented_image, augmented_coordinate[0], augmented_coordinate[1], (0,0,255), 2)

display_image(augmented_image_image_with_label)

# 모든 가능한 조합으로 대량 생산하기

In [ ]:
import traceback


original_image_dir = 'objects'
background_image_dir = 'backgrounds'
segmented_image_dir = 'segmentations'


for root, dirs, files in os.walk(original_image_dir):
    for f in files:
        for root2, dirs2, files2 in os.walk(background_image_dir):
            for f2 in files2:
                try:
                    original_image_path = os.path.join(root, f)
                    background_image_path = os.path.join(root2, f2)
                    segmented_image_path = original_image_path.replace('objects', 'segmentations').replace('.jpg', '.png')
                    call_create_augmented_image(original_image_path, background_image_path, segmented_image_path)
                except:
                    # 디버깅이 완료되고 나면 이곳을 주석처리 합시다
                    # traceback.print_exc()
                    print('error', original_image_path, background_image_path, segmented_image_path)


# 파일로 따로 빼서 검증해봅시다
- 위에 만든 함수를 py 파일로 저장하고 아래와 같이 실행해봅니다
- 그러면 이 파일만 대량으로 호출하면 대량 생산이 되겠네요

In [ ]:
! find /content/objects/bird | awk '{printf "python create_images.py %s %s %s\n", $1,$2,$3}' | bash

In [ ]:
! cat /content/VOCdevkit/VOC2012/ImageSets/Main/train.txt | awk '{printf "VOCdevkit/VOC2012/Annotations/%s.xml\n",$1}' | xargs grep "<name" | awk '{print $2}' | sed 's/<name>//g' | sed 's/<\/name>//g'  | grep -v hand | grep -v head | grep -v foot |  sort | uniq > classes.txt
! python create_image.py /content/objects/bird/284_bird_2009_000454.jpg /content/backgrounds/bottom_aeroplane_2009_002314.jpg /content/segmentations/bird/284_bird_2009_000454.png 

# 이렇게 해서 좋은 점이 무엇인가요?
- 증강 계획에 따라 조합을 만들기 좋습니다
  - 증강이 필요한 클래스 위주로 증강
  - 이미 실존 이미지가 많은 클래스까지 증강을 하면 클래스 분포 불균형이 해결되지 않겠지요

- 실행중 메모리 관리에 자신이 없을때 좋습니다
  - 증강 하다가 메모리 부족으로 죽는 경우를 줄일 수 있습니다
  - 프로세스의 시작과 종료가 클래스 단위로 발생하기 때문이지요
  - 메모리가 정말 부족하면 1장 단위로 증강하는 스크립트를 만들면 좀더 메모리 관리가 쉽습니다

- 이미지 증강 데이터세트의 관리가 쉬워집니다
  - 증강 이미지 데이터세트를 일일이 저장하기 어려워지고 데이터세트 관리가 깨지는 순간이 분명히 올겁니다 
  - 이 때 필요한게 무엇? DevOps? MLOps? 
  - 우리는 AugOps를 합시다 -> 이런 용어가 있는지는 모르겠지만 우리 과정에 맞게 이렇게 불러봅니다
  - 이미지 데이터세트를 관리하는것이 아니라 증강 파라미터와 호출 명령세트를 관리합니다

- 멀티 프로세싱을 하기 쉽습니다
  - 별도로 python 코드 내에서 멀티 프로세싱/멀티 스레딩을 구현하지 않으면 아무리 좋은 장비를 가지고 있어도 단일코어만 사용하게 됩니다
  - bash 스크립트를 이용하면 python 코드 내에서 멀티 프로세싱/멀티 스레딩을 구현하지 않아도 멀티코어를 쉽게 활용할 수 있습니다
  - AugOps를 하려고 해도 그 재현에 시간이 오래 걸리면 의미가 없겠지요, 이 때 필요한게 멀티 프로세싱

# Pre-trained 모델을 이용한 이미지 소스 생산
- 대량생산을 위한 준비가 다 되었습니다, 이제 원하는대로 호출만 하면 됩니다
- 그런데 혹시 원하는 이미지 소스가 부족한가요?

## COCO Unlabeled 데이터셋을 사용해봅시다
- http://images.cocodataset.org/zips/unlabeled2017.zip


## 여기에 Pre-grained 모델로 세그멘테이션 마스크를 생성해봅시다
- https://github.com/divamgupta/image-segmentation-keras


In [ ]:
# ! wget http://images.cocodataset.org/zips/unlabeled2017.zip
! wget http://images.cocodataset.org/zips/val2017.zip
! unzip val2017.zip

In [ ]:
! pip install --upgrade git+https://github.com/divamgupta/image-segmentation-keras

In [ ]:
! pip install tensorflow==2.4.1
! pip install keras==2.4.3
# ! pip install tensorflow-gpu==2.4.1

In [ ]:
display_image(cv2.imread('/content/val2017/000000005477.jpg'))

In [ ]:
from keras_segmentation.pretrained import pspnet_50_ADE_20K , pspnet_101_cityscapes, pspnet_101_voc12

model = pspnet_101_voc12() # load the pretrained model trained on Pascal VOC 2012 dataset

# load any of the 3 pretrained models

out = model.predict_segmentation(
    inp="/content/val2017/000000005477.jpg",
    out_fname="/content/000000005477.png")

In [ ]:
display_image(cv2.imread('/content/000000005477.png'))

## 세그멘테이션 마스크 색상
- 세그멘테이션 마스크상의 배경색이 검정색이 아니므로 약간 변경이 필요합니다
- 배경색의 BGR 코드는 [197, 215, 20] 입니다

In [ ]:
image = cv2.imread('/content/000000005477.png')
image[(image == [197, 215, 20]).all(axis=2)] = [0,0,0]

In [ ]:
cv2.imwrite('000000005477.png', image)
display_image(image)

# 완성되었으니 테스트 해볼까요?

In [ ]:
height, width, channel = cv2.imread('/content/backgrounds/bottom_aeroplane_2009_002314.jpg').shape

cv2.imwrite('/content/000000005477.png',  cv2.resize(cv2.imread('/content/000000005477.png'), (width, height)))
cv2.imwrite('/content/000000005477.jpg',  cv2.resize(cv2.imread('/content/val2017/000000005477.jpg'), (width, height)))

In [ ]:
cv2.imread('/content/backgrounds/bottom_aeroplane_2010_003259.jpg').shape
cv2.imread('/content/000000005477.png').shape

In [ ]:
augmented_image, augmented_coordinate = create_augmented_image( original_image_path = '/content/000000005477.jpg', \
    background_image_path = '/content/backgrounds/bottom_aeroplane_2009_002314.jpg', \
    segmented_image_path = '/content/000000005477.png')

display_image(augmented_image)
augmented_image_image_with_label = cv2.rectangle(augmented_image, augmented_coordinate[0], augmented_coordinate[1], (0,0,255), 2)

display_image(augmented_image_image_with_label)